In [ ]:
data = pd.DataFrame({
    'Income': ['Normal', 'High', 'Low', 'High', 'Low', 'High', 'Low', 'High', 'Low', 'High'],
    'Satisfaction': ['Yes', 'Yes', 'No', 'Yes', 'No', 'No', 'No', 'Yes', 'No', 'Yes']
})



In [ ]:
unique_income_values = data['Income'].unique()
print(unique_income_values)

['Normal' 'High' 'Low']


In [ ]:
income_satisfaction_counts = data.groupby('Income')['Satisfaction'].value_counts()
print(income_satisfaction_counts)

Income  Satisfaction
High    Yes             4
        No              1
Low     No              4
Normal  Yes             1
Name: count, dtype: int64


In [ ]:
first_income_value = data['Income'].iloc[0]  # İlk Income değerini al

# İlk değer dışında kalanları "Diğer" olarak değiştir
data['Income'] = data['Income'].apply(lambda x: 'Diğer' if x != first_income_value else x)

# Grupla ve say
income_satisfaction_counts = data.groupby('Income')['Satisfaction'].value_counts()
print(income_satisfaction_counts)
data.head()

Income  Satisfaction
Diğer   No              5
        Yes             4
Normal  Yes             1
Name: count, dtype: int64


,Income,Satisfaction
0,Normal,Yes
1,Diğer,Yes
2,Diğer,No
3,Diğer,Yes
4,Diğer,No


In [ ]:
data_copy = data.copy()  # DataFrame'in bir kopyasını oluştur

first_income_value = data_copy['Income'].iloc[0]

# İlk değer dışında kalanları "Diğer" olarak değiştir (kopya üzerinde)
data_copy['Income'] = data_copy['Income'].apply(lambda x: 'Diğer' if x != first_income_value else x)

# Grupla ve say (kopya üzerinde)
income_satisfaction_counts = data_copy.groupby('Income')['Satisfaction'].value_counts()
print(income_satisfaction_counts)

Income  Satisfaction
Diğer   No              5
        Yes             4
Normal  Yes             1
Name: count, dtype: int64


In [ ]:

# Grupla ve say
income_satisfaction_counts = data_copy.groupby('Income')['Satisfaction'].value_counts()

# Olasılıkları hesapla
probabilities = income_satisfaction_counts / income_satisfaction_counts.groupby(level=0).sum()

# Tabloya geçir
probability_table = probabilities.unstack(fill_value=0)
probability_table.columns = ['P(No)', 'P(Yes)']  # Sütun adlarını değiştir
probability_table = probability_table.sort_index()  # İndeks'e göre sırala
group_counts = data_copy['Income'].value_counts()

# Grup oranlarını hesapla
group_ratios = group_counts / len(data)

# Oranları olasılık tablosuna ekle
probability_table['Group Ratio'] = group_ratios

print(probability_table)

           P(No)    P(Yes)  Group Ratio
Income                                 
Diğer   0.555556  0.444444          0.9
Normal  0.000000  1.000000          0.1


In [ ]:
normal_yes_prob = probability_table.loc['Normal', 'P(Yes)']
diger_yes_prob = probability_table.loc['Diğer', 'P(Yes)']
normal_no_prob = probability_table.loc['Normal', 'P(No)']
diger_no_prob = probability_table.loc['Diğer', 'P(No)']
group_ratio_normal = probability_table.loc['Normal', 'Group Ratio']
group_ratio_diger = probability_table.loc['Diğer', 'Group Ratio']
print(normal_yes_prob)
print(diger_yes_prob)
print(normal_no_prob)
print(diger_no_prob)
print(group_ratio_normal)
print(group_ratio_diger)
# Formülü uygula
total_diff = abs(normal_yes_prob - diger_yes_prob) + abs(normal_no_prob - diger_no_prob)

print(f"Toplam fark: {total_diff}")

1.0
0.4444444444444444
0.0
0.5555555555555556
0.1
0.9
Toplam fark: 1.1111111111111112


In [ ]:
value = (2*total_diff*group_ratio_normal*group_ratio_diger)
print(value)

0.2


In [ ]:
import pandas as pd

def calculate_twoing_criterion(data, target_column='Satisfaction'):
    best_split_value = None
    best_criterion_value = -1  # Başlangıçta en düşük değeri ata

    for income_value in data['Income'].unique():
        # Veriyi kopyala ve gruplandır
        data_copy = data.copy()
        data_copy['Income'] = data_copy['Income'].apply(lambda x: 'Diğer' if x != income_value else x)

        # Olasılıkları hesapla
        income_satisfaction_counts = data_copy.groupby('Income')['Satisfaction'].value_counts()
        probabilities = income_satisfaction_counts / income_satisfaction_counts.groupby(level=0).sum()
        probability_table = probabilities.unstack(fill_value=0)
        probability_table.columns = ['P(No)', 'P(Yes)']

        # Grup oranlarını hesapla
        group_counts = data_copy['Income'].value_counts()
        group_ratios = group_counts / len(data_copy)
        probability_table['Group Ratio'] = group_ratios

        # Uygunluk ölçütünü hesapla
        normal_yes_prob = probability_table.loc[income_value, 'P(Yes)']  # income_value kullan
        diger_yes_prob = probability_table.loc['Diğer', 'P(Yes)']
        normal_no_prob = probability_table.loc[income_value, 'P(No)']  # income_value kullan
        diger_no_prob = probability_table.loc['Diğer', 'P(No)']

        total_diff = abs(normal_yes_prob - diger_yes_prob) + abs(normal_no_prob - diger_no_prob)
        criterion_value = (2 * total_diff * group_ratios[income_value] * group_ratios['Diğer'])  # Güncellenmiş formül
        print(criterion_value , ' '  , income_value)
        # En iyi değeri güncelle
        if criterion_value > best_criterion_value:
            best_criterion_value = criterion_value
            best_split_value = income_value

    return best_split_value, best_criterion_value

# Örnek veri
data = pd.DataFrame({
    'Income': ['Normal', 'High', 'Low', 'High', 'Low', 'High', 'Low', 'High', 'Low', 'High'],
    'Satisfaction': ['Yes', 'Yes', 'No', 'Yes', 'No', 'Yes', 'No', 'Yes', 'No', 'Yes']
})

# Fonksiyonu çağır ve sonuçları yazdır
best_split, best_criterion = calculate_twoing_criterion(data)
print(f"En iyi bölme değeri: {best_split}")
print(f"En iyi uygunluk ölçütü değeri: {best_criterion}")

0.16   Normal
0.8   High
0.96   Low
En iyi bölme değeri: Low
En iyi uygunluk ölçütü değeri: 0.96


In [ ]:
import pandas as pd
import itertools

def calculate_twoing_criterion(data, target_column):
    """
    Calculates the twoing criterion for splits across multiple columns and prints all calculations.

    Args:
        data: The input DataFrame.
        target_column: The target column for calculating probabilities.

    Returns:
        A tuple containing the best split combination and the corresponding criterion value.
    """
    best_split_combination = None
    best_criterion_value = -1

    # Get all columns except the target column
    split_columns = [col for col in data.columns if col != target_column]

    # Generate all possible combinations of split columns
    for r in range(1, len(split_columns) + 1):  # Iterate through combinations of 1, 2, ... columns
        for split_column_combination in itertools.combinations(split_columns, r):
            # Generate all possible split value combinations for the selected columns
            unique_values_per_column = [data[col].unique() for col in split_column_combination]
            for split_value_combination in itertools.product(*unique_values_per_column):
                # Create a copy of the data and apply the split
                data_copy = data.copy()
                split_condition = pd.Series(True, index=data_copy.index)  # Initialize to True
                for col, split_value in zip(split_column_combination, split_value_combination):
                    split_condition &= (data_copy[col] == split_value)  # Update condition

                data_copy['Group'] = 'Diğer'  # Assign 'Diğer' to all rows initially
                data_copy.loc[split_condition, 'Group'] = 'Split Group'  # Assign 'Split Group' to matching rows

                # Calculate probabilities and group ratios
                group_counts = data_copy.groupby('Group')[target_column].value_counts()
                probabilities = group_counts / group_counts.groupby(level=0).sum()
                probability_table = probabilities.unstack(fill_value=0)

                if len(probability_table.columns) == 1:
                    probability_table['dummy'] = 0

                probability_table.columns = ['P(Hayır)', 'P(Evet)'][:len(probability_table.columns)]
                group_ratios = data_copy['Group'].value_counts() / len(data_copy)
                probability_table['Group Ratio'] = group_ratios

                # Calculate the twoing criterion
                try:
                    normal_yes_prob = probability_table.loc['Split Group', 'P(Evet)']
                    diger_yes_prob = probability_table.loc['Diğer', 'P(Evet)']
                    normal_no_prob = probability_table.loc['Split Group', 'P(Hayır)']
                    diger_no_prob = probability_table.loc['Diğer', 'P(Hayır)']

                except KeyError:
                    continue

                total_diff = abs(normal_yes_prob - diger_yes_prob) + abs(normal_no_prob - diger_no_prob)
                criterion_value = (2 * total_diff * group_ratios['Split Group'] * group_ratios['Diğer'])

                # Print the criterion value for the current split combination
                print(f"For split combination: {split_column_combination}, values: {split_value_combination}, criterion_value: {criterion_value}")


                # Update the best split combination
                if criterion_value > best_criterion_value:
                    best_criterion_value = criterion_value
                    best_split_combination = (split_column_combination, split_value_combination)

    return best_split_combination, best_criterion_value

data = pd.DataFrame({
       'Income': ['Normal', 'Büyük', 'Küçük', 'Büyük', 'Küçük', 'Büyük', 'Küçük', 'Büyük', 'Küçük', 'Büyük', 'Küçük'],
       'Education': ['Orta', 'İlk', 'İlk', 'Orta', 'Orta', 'Lise', 'Lise', 'Orta', 'Orta', 'Lise', 'Lise'],
       'Sector': ['Bilişim', 'Bilişim', 'İnşaat', 'İnşaat', 'İnşaat', 'İnşaat', 'İnşaat', 'Bilişim', 'Bilişim', 'Bilişim', 'Bilişim'],
       'Satisfaction': ['Evet', 'Evet', 'Evet', 'Evet', 'Evet', 'Evet', 'Evet', 'Hayır', 'Hayır', 'Hayır', 'Hayır']
   })

target_column = 'Satisfaction'

# Fonksiyonu çağır ve sonuçları yazdır
best_split, best_criterion = calculate_twoing_criterion(data,target_column)
print(f"En iyi bölme değeri: {best_split}")
print(f"En iyi uygunluk ölçütü değeri: {best_criterion}")

For split combination: ('Income',), values: ('Normal',), criterion_value: 0.1322314049586777
For split combination: ('Income',), values: ('Büyük',), criterion_value: 0.06611570247933884
For split combination: ('Income',), values: ('Küçük',), criterion_value: 0.06611570247933884
For split combination: ('Education',), values: ('Orta',), criterion_value: 0.06611570247933884
For split combination: ('Education',), values: ('İlk',), criterion_value: 0.2644628099173554
For split combination: ('Education',), values: ('Lise',), criterion_value: 0.19834710743801656
For split combination: ('Sector',), values: ('Bilişim',), criterion_value: 0.6611570247933884
For split combination: ('Sector',), values: ('İnşaat',), criterion_value: 0.6611570247933884
For split combination: ('Income', 'Education'), values: ('Normal', 'Orta'), criterion_value: 0.1322314049586777
For split combination: ('Income', 'Education'), values: ('Büyük', 'Orta'), criterion_value: 0.09917355371900827
For split combination: ('In

In [ ]:
import pandas as pd

def calculate_twoing_criterion(data, target_column):
    """
    Calculates the twoing criterion for splits within each column individually.

    Args:
        data: The input DataFrame.
        target_column: The target column for calculating probabilities.

    Returns:
        A tuple containing the best split combination and the corresponding criterion value.
    """
    best_split_combination = None
    best_criterion_value = -1

    # Get all columns except the target column
    split_columns = [col for col in data.columns if col != target_column]

    # Iterate through each split column individually
    for col in split_columns:
        for split_value in data[col].unique():
            # Create a copy of the data and apply the split
            data_copy = data.copy()
            data_copy[col] = data_copy[col].apply(lambda x: 'Other' if x != split_value else x)

            # Calculate probabilities and group ratios
            group_counts = data_copy.groupby(col)[target_column].value_counts()
            probabilities = group_counts / group_counts.groupby(level=0).sum()
            probability_table = probabilities.unstack(fill_value=0)

            if len(probability_table.columns) == 1:
                probability_table['dummy'] = 0

            probability_table.columns = ['P(No)', 'P(Yes)'][:len(probability_table.columns)]
            group_ratios = data_copy[col].value_counts() / len(data_copy)
            probability_table['Group Ratio'] = group_ratios

            # Calculate the twoing criterion
            try:
                normal_yes_prob = probability_table.loc[split_value, 'P(Yes)']
                diger_yes_prob = probability_table.loc['Other', 'P(Yes)']
                normal_no_prob = probability_table.loc[split_value, 'P(No)']
                diger_no_prob = probability_table.loc['Other', 'P(No)']

            except KeyError:
                continue

            total_diff = abs(normal_yes_prob - diger_yes_prob) + abs(normal_no_prob - diger_no_prob)
            criterion_value = (2 * total_diff * group_ratios[split_value] * group_ratios['Other'])

            # Print the criterion value for the current split combination
            print(f"For split combination: ({col},), values: ({split_value},), criterion_value: {criterion_value}")


            # Update the best split combination
            if criterion_value > best_criterion_value:
                best_criterion_value = criterion_value
                best_split_combination = ((col,), (split_value,))

    return best_split_combination, best_criterion_value

data = pd.DataFrame({
       'Income': ['Normal', 'Büyük', 'Küçük', 'Büyük', 'Küçük', 'Büyük', 'Küçük', 'Büyük', 'Küçük', 'Büyük', 'Küçük'],
       'Education': ['Orta', 'İlk', 'İlk', 'Orta', 'Orta', 'Lise', 'Lise', 'Orta', 'Orta', 'Lise', 'Lise'],
       'Sector': ['Bilişim', 'Bilişim', 'İnşaat', 'İnşaat', 'İnşaat', 'İnşaat', 'İnşaat', 'Bilişim', 'Bilişim', 'Bilişim', 'Bilişim'],
       'Satisfaction': ['Evet', 'Evet', 'Evet', 'Evet', 'Evet', 'Evet', 'Evet', 'Hayır', 'Hayır', 'Hayır', 'Hayır']
   })

target_column = 'Satisfaction'

# Fonksiyonu çağır ve sonuçları yazdır
best_split, best_criterion = calculate_twoing_criterion(data,target_column)
print(f"En iyi bölme değeri: {best_split}")
print(f"En iyi uygunluk ölçütü değeri: {best_criterion}")

For split combination: (Income,), values: (Normal,), criterion_value: 0.1322314049586777
For split combination: (Income,), values: (Büyük,), criterion_value: 0.06611570247933884
For split combination: (Income,), values: (Küçük,), criterion_value: 0.06611570247933884
For split combination: (Education,), values: (Orta,), criterion_value: 0.06611570247933884
For split combination: (Education,), values: (İlk,), criterion_value: 0.2644628099173554
For split combination: (Education,), values: (Lise,), criterion_value: 0.19834710743801656
For split combination: (Sector,), values: (Bilişim,), criterion_value: 0.6611570247933884
For split combination: (Sector,), values: (İnşaat,), criterion_value: 0.6611570247933884
En iyi bölme değeri: (('Sector',), ('Bilişim',))
En iyi uygunluk ölçütü değeri: 0.6611570247933884


In [ ]:
import pandas as pd


def build_tree(data, target_column):
    """
    Builds a tree structure where the selected best split becomes a root node, and its opposite value
    becomes another root node with the original split as its child.

    Args:
        data: The input DataFrame.
        target_column: The target column for calculating probabilities.

    Returns:
        A dictionary representing the tree structure.
    """

    best_split_combination, best_criterion_value = calculate_twoing_criterion(data, target_column)

    if best_split_combination is None:
        return None  # No suitable split found

    split_column = best_split_combination[0][0]  # Get the split column
    split_value = best_split_combination[1][0]  # Get the split value

    # Get the opposite value (assuming binary splits)
    opposite_value = data[split_column].unique()[data[split_column].unique() != split_value][0]

    # Build the tree structure
    tree = {
        opposite_value: {
            split_value: {}  # You can add further logic here to continue building the tree recursively
        }
    }

    return tree

def calculate_twoing_criterion(data, target_column):
    """
    Calculates the twoing criterion for splits within each column individually.

    Args:
        data: The input DataFrame.
        target_column: The target column for calculating probabilities.

    Returns:
        A tuple containing the best split combination and the corresponding criterion value.
    """
    best_split_combination = None
    best_criterion_value = -1

    # Get all columns except the target column
    split_columns = [col for col in data.columns if col != target_column]

    # Iterate through each split column individually
    for col in split_columns:
        for split_value in data[col].unique():
            # Create a copy of the data and apply the split
            data_copy = data.copy()
            data_copy[col] = data_copy[col].apply(lambda x: 'Diğer' if x != split_value else x)

            # Calculate probabilities and group ratios
            group_counts = data_copy.groupby(col)[target_column].value_counts()
            probabilities = group_counts / group_counts.groupby(level=0).sum()
            probability_table = probabilities.unstack(fill_value=0)

            if len(probability_table.columns) == 1:
                probability_table['dummy'] = 0

            probability_table.columns = ['P(Hayır)', 'P(Evet)'][:len(probability_table.columns)]
            group_ratios = data_copy[col].value_counts() / len(data_copy)
            probability_table['Group Ratio'] = group_ratios

            # Calculate the twoing criterion
            try:
                normal_yes_prob = probability_table.loc[split_value, 'P(Evet)']
                diger_yes_prob = probability_table.loc['Diğer', 'P(Evet)']
                normal_no_prob = probability_table.loc[split_value, 'P(Hayır)']
                diger_no_prob = probability_table.loc['Diğer', 'P(Hayır)']

            except KeyError:
                continue

            total_diff = abs(normal_yes_prob - diger_yes_prob) + abs(normal_no_prob - diger_no_prob)
            criterion_value = (2 * total_diff * group_ratios[split_value] * group_ratios['Diğer'])

            # Print the criterion value for the current split combination
            print(f"For split combination: ({col},), values: ({split_value},), criterion_value: {criterion_value}")


            # Update the best split combination
            if criterion_value > best_criterion_value:
                best_criterion_value = criterion_value
                best_split_combination = ((col,), (split_value,))

    return best_split_combination, best_criterion_value


data = pd.DataFrame({
       'Income': ['Normal', 'Büyük', 'Küçük', 'Büyük', 'Küçük', 'Büyük', 'Küçük', 'Büyük', 'Küçük', 'Büyük', 'Küçük'],
       'Education': ['Orta', 'İlk', 'İlk', 'Orta', 'Orta', 'Lise', 'Lise', 'Orta', 'Orta', 'Lise', 'Lise'],
       'Sector': ['Bilişim', 'Bilişim', 'İnşaat', 'İnşaat', 'İnşaat', 'İnşaat', 'İnşaat', 'Bilişim', 'Bilişim', 'Bilişim', 'Bilişim'],
       'Satisfaction': ['Evet', 'Evet', 'Evet', 'Evet', 'Evet', 'Evet', 'Evet', 'Hayır', 'Hayır', 'Hayır', 'Hayır']
   })

target_column = 'Satisfaction'

# Fonksiyonu çağır ve sonuçları yazdır

tree = build_tree(data, 'Satisfaction')
print(tree)

For split combination: (Income,), values: (Normal,), criterion_value: 0.1322314049586777
For split combination: (Income,), values: (Büyük,), criterion_value: 0.06611570247933884
For split combination: (Income,), values: (Küçük,), criterion_value: 0.06611570247933884
For split combination: (Education,), values: (Orta,), criterion_value: 0.06611570247933884
For split combination: (Education,), values: (İlk,), criterion_value: 0.2644628099173554
For split combination: (Education,), values: (Lise,), criterion_value: 0.19834710743801656
For split combination: (Sector,), values: (Bilişim,), criterion_value: 0.6611570247933884
For split combination: (Sector,), values: (İnşaat,), criterion_value: 0.6611570247933884
{'İnşaat': {'Bilişim': {}}}


In [ ]:
import pandas as pd
import graphviz

def calculate_twoing_criterion(data, target_column):
    """
    Calculates the twoing criterion for splits within each column individually.

    Args:
        data: The input DataFrame.
        target_column: The target column for calculating probabilities.

    Returns:
        A tuple containing the best split combination and the corresponding criterion value.
    """
    best_split_combination = None
    best_criterion_value = -1

    # Get all columns except the target column
    split_columns = [col for col in data.columns if col != target_column]

    # Iterate through each split column individually
    for col in split_columns:
        for split_value in data[col].unique():
            # Create a copy of the data and apply the split
            data_copy = data.copy()
            data_copy[col] = data_copy[col].apply(lambda x: split_value if x == split_value else 'Diğer')

            # Calculate probabilities and group ratios
            group_counts = data_copy.groupby(col)[target_column].value_counts()
            probabilities = group_counts / group_counts.groupby(level=0).sum()
            probability_table = probabilities.unstack(fill_value=0)

            if len(probability_table.columns) == 1:
                probability_table['dummy'] = 0

            probability_table.columns = ['P(Hayır)', 'P(Evet)'][:len(probability_table.columns)]
            group_ratios = data_copy[col].value_counts() / len(data_copy)
            probability_table['Group Ratio'] = group_ratios

            # Calculate the twoing criterion
            try:
                normal_yes_prob = probability_table.loc[split_value, 'P(Evet)']
                diger_yes_prob = probability_table.loc['Diğer', 'P(Evet)']
                normal_no_prob = probability_table.loc[split_value, 'P(Hayır)']
                diger_no_prob = probability_table.loc['Diğer', 'P(Hayır)']

            except KeyError:
                continue

            total_diff = abs(normal_yes_prob - diger_yes_prob) + abs(normal_no_prob - diger_no_prob)
            criterion_value = (2 * total_diff * group_ratios[split_value] * group_ratios['Diğer'])

            # Print the criterion value for the current split combination
            print(f"For split combination: ({col},), values: ({split_value},), criterion_value: {criterion_value}")

            # Update the best split combination
            if criterion_value > best_criterion_value:
                best_criterion_value = criterion_value
                best_split_combination = (col, split_value)

    return best_split_combination, best_criterion_value

def build_tree(data, target_column):
    """
    Builds a decision tree recursively using the twoing criterion.
    Removes the used column at each split.

    Args:
        data: The input DataFrame.
        target_column: The target column for classification.

    Returns:
        A dictionary representing the tree structure.
    """
    # If all the values in target column are the same, return the class as the leaf
    if len(data[target_column].unique()) == 1:
        return data[target_column].iloc[0]

    best_split_combination, best_criterion_value = calculate_twoing_criterion(data, target_column)

    # If no valid split is found, return the majority class as the leaf node
    if best_split_combination is None:
        return data[target_column].value_counts().idxmax()

    # Extract split column and value
    split_column = best_split_combination[0]
    split_value = best_split_combination[1]

    # Split the data into two subsets
    left_split = data[data[split_column] == split_value]
    right_split = data[data[split_column] != split_value]

    # Remove the split column from both subsets
    left_split = left_split.drop(columns=[split_column])
    right_split = right_split.drop(columns=[split_column])

    # If all values in left or right split are the same, make it a leaf node
    if len(left_split[target_column].unique()) == 1:
        left_leaf = left_split[target_column].iloc[0]
    else:
        left_leaf = build_tree(left_split, target_column)

    if len(right_split[target_column].unique()) == 1:
        right_leaf = right_split[target_column].iloc[0]
    else:
        right_leaf = build_tree(right_split, target_column)

    # Build the tree recursively
    tree = {
        split_column: {
            split_value: left_leaf,  # Leaf node if it's the same class
            'Diğer': right_leaf  # Leaf node if it's the same class
        }
    }

    return tree

def visualize_tree(tree, parent_name="root"):
    """
    Recursively visualizes the decision tree.

    Args:
        tree: The decision tree as a dictionary.
        parent_name: The name of the parent node for visualization purposes.

    Returns:
        A Graphviz object representing the decision tree.
    """
    dot = graphviz.Digraph(comment='Decision Tree')

    def add_nodes_edges(tree, parent_name):
        for key, value in tree.items():
            # Add the current node
            current_node_name = f"{parent_name}_{key}"
            dot.node(current_node_name, label=str(key))

            # Create an edge between parent and current node
            dot.edge(parent_name, current_node_name)

            # Recursively add child nodes
            if isinstance(value, dict):
                add_nodes_edges(value, current_node_name)
            else:
                dot.node(f"{current_node_name}_leaf", label=str(value), shape="ellipse")
                dot.edge(current_node_name, f"{current_node_name}_leaf")

    # Start the recursion
    add_nodes_edges(tree, parent_name)

    return dot


# Example DataFrame
data = pd.DataFrame({
    'Income': ['Normal', 'Büyük', 'Küçük', 'Büyük', 'Küçük', 'Büyük', 'Küçük', 'Büyük', 'Küçük', 'Büyük', 'Küçük'],
    'Education': ['Orta', 'İlk', 'İlk', 'Orta', 'Orta', 'Lise', 'Lise', 'Orta', 'Orta', 'Lise', 'Lise'],
    'Sector': ['Bilişim', 'Bilişim', 'İnşaat', 'İnşaat', 'İnşaat', 'İnşaat', 'İnşaat', 'Bilişim', 'Bilişim', 'Bilişim', 'Bilişim'],
    'Satisfaction': ['Evet', 'Evet', 'Evet', 'Evet', 'Evet', 'Evet', 'Evet', 'Hayır', 'Hayır', 'Hayır', 'Hayır']
})

# Define the target column
target_column = 'Satisfaction'

# Call the function and print the tree
tree = build_tree(data, target_column)
print(tree)

# Visualize the tree
decision_tree_graph = visualize_tree(tree)

# Render the tree and save it as a PNG image
decision_tree_graph.render("decision_tree", format="png", view=False)  # "decision_tree.png" will be saved and not displayed


For split combination: (Income,), values: (Normal,), criterion_value: 0.1322314049586777
For split combination: (Income,), values: (Büyük,), criterion_value: 0.06611570247933884
For split combination: (Income,), values: (Küçük,), criterion_value: 0.06611570247933884
For split combination: (Education,), values: (Orta,), criterion_value: 0.06611570247933884
For split combination: (Education,), values: (İlk,), criterion_value: 0.2644628099173554
For split combination: (Education,), values: (Lise,), criterion_value: 0.19834710743801656
For split combination: (Sector,), values: (Bilişim,), criterion_value: 0.6611570247933884
For split combination: (Sector,), values: (İnşaat,), criterion_value: 0.6611570247933884
For split combination: (Income,), values: (Normal,), criterion_value: 0.4444444444444445
For split combination: (Income,), values: (Büyük,), criterion_value: 0.0
For split combination: (Income,), values: (Küçük,), criterion_value: 0.4444444444444444
For split combination: (Education

'decision_tree.png'

In [ ]:
import pandas as pd
import graphviz

def calculate_twoing_criterion(data, target_column):
    """
    Calculates the twoing criterion for splits within each column individually.
    """
    best_split_combination = None
    best_criterion_value = -1

    split_columns = [col for col in data.columns if col != target_column]

    for col in split_columns:
        for split_value in data[col].unique():
            data_copy = data.copy()
            data_copy[col] = data_copy[col].apply(lambda x: split_value if x == split_value else 'Diğer')

            group_counts = data_copy.groupby(col)[target_column].value_counts()
            probabilities = group_counts / group_counts.groupby(level=0).sum()
            probability_table = probabilities.unstack(fill_value=0)

            if len(probability_table.columns) == 1:
                probability_table['dummy'] = 0

            probability_table.columns = ['P(Hayır)', 'P(Evet)'][:len(probability_table.columns)]
            group_ratios = data_copy[col].value_counts() / len(data_copy)
            probability_table['Group Ratio'] = group_ratios

            try:
                normal_yes_prob = probability_table.loc[split_value, 'P(Evet)']
                diger_yes_prob = probability_table.loc['Diğer', 'P(Evet)']
                normal_no_prob = probability_table.loc[split_value, 'P(Hayır)']
                diger_no_prob = probability_table.loc['Diğer', 'P(Hayır)']
            except KeyError:
                continue

            total_diff = abs(normal_yes_prob - diger_yes_prob) + abs(normal_no_prob - diger_no_prob)
            criterion_value = (2 * total_diff * group_ratios[split_value] * group_ratios['Diğer'])

            if criterion_value > best_criterion_value:
                best_criterion_value = criterion_value
                best_split_combination = (col, split_value)

    return best_split_combination, best_criterion_value

def build_tree(data, target_column):
    """
    Builds a decision tree recursively using the twoing criterion.
    """
    if len(data[target_column].unique()) == 1:
        return data[target_column].iloc[0]

    best_split_combination, best_criterion_value = calculate_twoing_criterion(data, target_column)

    if best_split_combination is None:
        return data[target_column].value_counts().idxmax()

    split_column = best_split_combination[0]
    split_value = best_split_combination[1]

    left_split = data[data[split_column] == split_value]
    right_split = data[data[split_column] != split_value]

    if len(left_split) == 0 or len(right_split) == 0:
        return data[target_column].value_counts().idxmax()

    left_leaf = build_tree(left_split, target_column)
    right_leaf = build_tree(right_split, target_column)

    tree = {split_column: {split_value: left_leaf, 'Diğer': right_leaf}}
    return tree

def visualize_tree(tree):
    """
    Recursively visualizes the decision tree with labeled branches.
    """
    dot = graphviz.Digraph(comment="Decision Tree")

    def add_nodes_edges(subtree, parent_name, parent_label=""):
        for key, value in subtree.items():
            # Düğümün etiketi
            edge_label = str(key)

            if isinstance(value, dict):
                child_name = f"{parent_name}_{key}"
                dot.node(child_name, label=key)
                dot.edge(parent_name, child_name, label=edge_label)
                add_nodes_edges(value, child_name)
            else:
                leaf_name = f"{parent_name}_{key}_leaf"
                dot.node(leaf_name, label=str(value), shape="ellipse")
                dot.edge(parent_name, leaf_name, label=edge_label)

    root_name = "root"
    dot.node(root_name, label="root")
    add_nodes_edges(tree, root_name)

    return dot


# Load data from CSV
data = pd.read_csv('/content/twoing_algorithms.csv')

# Set the target column
target_column = 'Satisfaction'

# Build the decision tree
tree = build_tree(data, target_column)
print(tree)

# Visualize the decision tree
decision_tree_graph = visualize_tree(tree)
decision_tree_graph.render("decision_tree", format="png", view=True)  # Save as PNG and open


{'Sector': {'Bilişim': {'Income': {'Normal': 'Evet', 'Diğer': {'Education': {'İlk': 'Evet', 'Diğer': 'Hayır'}}}}, 'Diğer': 'Evet'}}


'decision_tree.png'